# Louis Long Notebook

### Emil studid & Louis studid, teamname 

#### Table of contents:
1. Exploratory data analysis
2. Models/Predictors
    - Model 1
    - Model 2
    - model ...
3. Feature Engineering 
    - Lime
    - feature importance
    - PDP
4. Model Interpretations
5. Improved models (possibly)



# ___________ _0. Setup_ ___________

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn.metrics as metrics
import sklearn.ensemble as ensemble
import optuna
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, mean_squared_error, mean_squared_log_error
from verstack import LGBMTuner

#from pandas_profiling import ProfileReport

In [4]:
def writeResultToFile(test_data, pred_data, nameOfFile='namelessSubmission'):
    submission = pd.DataFrame()
    submission['id'] = test_data['store_id']
    submission['predicted'] = np.asarray(pred_data)
    submission.to_csv('submissionFiles/'+ nameOfFile+'.csv', index=False)
    

In [5]:
def rmsle(y_true, y_pred):
    return metrics.mean_squared_log_error(y_true, y_pred)**0.5

# ___________ _1. Exploratory Data Analysis_ ___________


### EDA Notes
- [ ] Search domain knowledge
- [ ] Check if the data is intuitive
- [ ] Understand how the data was generated
- [ ] Explore individual features
    - [x] Agencies
    - [x] stores with 0 revenue
    - [x] food and drink stores and grovery stores
- [ ] Explore pairs and groups
    - [x] Store type vs revenue
- [ ] Clean up features
    - [x] remove 2016
    - [x] remove outliers
    - [x] remove 0 revenue rows


In [ ]:
stores_train = pd.read_csv('data/stores_train.csv')


In [ ]:
stores_train.head()

In [ ]:
stores_train.info()

In [ ]:
stores_train.describe()

In [ ]:
#report = ProfileReport(stores_train)
#report

Explore revenue based on store type

In [ ]:
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
stores_with_hierarchy = stores_train.merge(plaace_hierarchy, how='left', on='plaace_hierarchy_id')

In [ ]:
plt.figure(figsize=(20,6))
plt.gcf().set_dpi(600)
plt.xticks(rotation=90)
sns.violinplot(x='lv2_desc',y='revenue',data=stores_with_hierarchy).set_title("Revenue on store type")
plt.show()

further exploration of agencie store type

In [ ]:
stores_with_hierarchy[stores_with_hierarchy["lv2_desc"]=="Agencies"]

Further exploration of "Food and drink" type stores


In [ ]:
plt.figure(figsize=(10,5))
sns.violinplot(x='lv3_desc',y='revenue',data=stores_with_hierarchy[stores_with_hierarchy["lv2_desc"]=="Food and drinks"]).set_title("Food and drinks violin plot")
plt.xticks(rotation=90)
plt.show()

Explore retailers with 0 revenue

In [ ]:
stores_with_hierarchy[stores_with_hierarchy["revenue"]==0.0].describe()

All retailers and their cooresponding revenue, the plot is to visually check for outliers, clearly there are som outliers as can be seen in the plot below

In [ ]:
plt.plot(stores_train['store_id'], stores_train['revenue'],'o')
plt.figure(figsize=(30,5))
plt.show()

### Cleaning

#### Remove columns function - example: year is a const value and has no effect on the end result

In [6]:
def remove_columns(dataSet, columns):
    for column in columns:
        dataSet.drop(column, axis=1, inplace=True)


In [ ]:
remove_columns(stores_train,['year'])
stores_train.head()

#### Remove retailers with 0 revenue function - might be handy

In [7]:
def remove_retailers_with_0_revenue(dataSet):
    dataSet.drop(dataSet[dataSet['revenue']==0.0].index, inplace=True)

In [ ]:
#remove_retailers_with_0_revenue(stores_train)

#### Removing outliers

Plotting all retailers based on storetype before and after trimming to confirm that outliers actually has been removed

In [ ]:
for store_type in stores_with_hierarchy['lv2_desc'].unique():
    plt.figure(figsize=(12,2))
    sns.violinplot(x='lv3_desc',y='revenue',data=stores_with_hierarchy[stores_with_hierarchy["lv2_desc"]==store_type]).set_title(f"{store_type} violin plot")
    plt.show()

Remove-outliers-function for the relationship between store type and revenue

In [ ]:
def quantile_storeType_vs_revenue(stores, lower, upper):
    for store_type in stores['plaace_hierarchy_id'].unique():
        data = stores[stores['plaace_hierarchy_id']==store_type]
        upper_treshold = data['revenue'].quantile(upper)
        lower_treshold = data['revenue'].quantile(lower)
        stores.drop(stores[(stores['plaace_hierarchy_id']==store_type) & (stores['revenue']>upper_treshold)].index, inplace=True)
        stores.drop(stores[(stores['plaace_hierarchy_id']==store_type) & (stores['revenue']<lower_treshold)].index, inplace=True)
    

In [ ]:
quantile_storeType_vs_revenue(stores_train)

Plot after removing outliers

you can see in the plot below that the outliers has been removed


In [ ]:
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
stores_with_hierarchy = stores_train.merge(plaace_hierarchy, how='left', on='plaace_hierarchy_id')
for store_type in stores_with_hierarchy['lv2_desc'].unique():
    plt.figure(figsize=(12,2))
    sns.violinplot(x='lv3_desc',y='revenue',data=stores_with_hierarchy[stores_with_hierarchy["lv2_desc"]==store_type]).set_title(f"{store_type} violin plot")
    plt.show()

#### comparing test set to training set

In [ ]:
stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')

comparing coordinates

In [ ]:
plt.figure(figsize=(16,9), dpi=600)
plt.scatter(stores_train['lon'],stores_train['lat'], label="traing",color='red')
plt.scatter(stores_test['lon'], stores_test['lat'], alpha=0.2, label="test", color="blue")
plt.legend(fontsize=10,ncol=2)
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.grid()
plt.show()



In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(projection='3d')
ax1.scatter(stores_train['lat'],stores_train['lon'],stores_train['revenue'])
ax1.set_xlabel('Lat')
ax1.set_ylabel('Lon')
ax1.set_zlabel('Revenue')
plt.show()


#### Examine whether a store occurs in multiple datasets 

In [ ]:
def stores_that_are_in_both_sets(df1, df2):
    
    duplicate_set = pd.merge(df1,df2, how='inner', on='store_name')
    return duplicate_set

stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')
stores_extra = pd.read_csv('data/stores_extra.csv')

dup = stores_that_are_in_both_sets(stores_test, stores_train)
dup.describe()

#

# 2.___________ _Machine Learning Models and Predictions_ ___________


## Louis modeller

## _____ LightGBM _____

Helper functions

In [8]:
def convert_DType_LGBM(dFrame):
    X = pd.DataFrame()
    for col_name in dFrame:
        if dFrame[col_name].dtypes == 'object':
            X[col_name] = dFrame[col_name].astype('category')
        else:
            X[col_name] = dFrame[col_name]
    
    return X


In [ ]:
stores_train = pd.read_csv('data/stores_train.csv')

# select prefered columns
remove_columns(stores_train, ['store_id','year','store_name','sales_channel_name','grunnkrets_id','address'])

# Divide data into train and test set
temp_x = stores_train.drop('revenue', axis=1)
temp_y = stores_train['revenue']

_, x_test, _, y_true = train_test_split(temp_x, temp_y, test_size=0.20, random_state=1)


# Preprocess/Clean data
quantile_storeType_vs_revenue(stores_train,0.02, 0.86)
#remove_retailers_with_0_revenue(stores_train)

# Divide data into x and y train, and test data for submission
x_train = stores_train.drop('revenue', axis=1)
y_train = stores_train['revenue']


# Convert from object type to numerical
x_train = convert_DType_LGBM(x_train)
x_test = convert_DType_LGBM(x_test)

In [ ]:
x_test.head()

Make model and train

In [ ]:
# Model
LGBM =LGBMRegressor()
#LGBM = lgb.LGBMRegressor(reg_alpha=0.2739452484147049, reg_lambda=9.205287883979267, colsample_bytree= 0.6, subsample=0.8, learning_rate= 0.014, max_depth= 100, num_leaves=449, min_child_samples=242)
# fitting
LGBM.fit(x_train, y_train)


Test LightGBM

In [ ]:
# predicting the training data set as a pin pointer
pred = LGBM.predict(x_test)
for i in range(len(pred)):
    if pred[i] < 0.0:
        print(i)
        pred[i] = 0.0
print(rmsle(y_true,pred))

In [ ]:
# Previous rmse scores gave the following kaggle scores:
# - 0.74281469137304 rmse resulted in: 0.75490 on kaggle

Autotesting percentile cut

In [ ]:
for i in range(0,10,1):
    stores_train = pd.read_csv('data/stores_train.csv')

    # select prefered columns
    remove_columns(stores_train, ['store_id','year','store_name','sales_channel_name','grunnkrets_id','address'])

    # Divide data into train and test set
    temp_x = stores_train.drop('revenue', axis=1)
    temp_y = stores_train['revenue']

    _, x_test, _, y_true = train_test_split(temp_x, temp_y, test_size=0.20, random_state=2)


    # Preprocess/Clean data
    quantile_storeType_vs_revenue(stores_train,i*0.01, 0.86)
    #remove_retailers_with_0_revenue(stores_train)

    # Divide data into x and y train, and test data for submission
    x_train = stores_train.drop('revenue', axis=1)
    y_train = stores_train['revenue']


    # Convert from object type to numerical
    x_train = convert_DType_LGBM(x_train)
    x_test = convert_DType_LGBM(x_test)
    # Model
    LGBM = lgb.LGBMRegressor(reg_alpha=0.2739452484147049, reg_lambda=9.205287883979267, colsample_bytree= 0.6, subsample=0.8, learning_rate= 0.014, max_depth= 100, num_leaves=449, min_child_samples=242)

    # fitting
    LGBM.fit(x_train, y_train)
    # predicting the test data
    pred = LGBM.predict(x_test)
    for j in range(len(pred)):
        if pred[j] < 0.0:
            pred[j] = 0.0
    print(f"for upper limit {round((i*0.01), 2)}, rmsle = {round(rmsle(y_true,pred), 4)}")

Predict test and submit

In [ ]:
stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')
test = stores_test.copy()

# Preprocess/Clean data
remove_columns(stores_train, ['store_id','year','store_name','sales_channel_name','grunnkrets_id','address'])
remove_columns(test, ['store_id','year','store_name','sales_channel_name','grunnkrets_id','address'])
quantile_storeType_vs_revenue(stores_train,0.02, 0.86)
#remove_retailers_with_0_revenue(stores_train)

# Divide data into x and y train, and test data for submission
x_train = stores_train.drop('revenue', axis=1)
y_train = stores_train['revenue']

# Convert from object type to numerical
x_train = convert_DType_LGBM(x_train)
test = convert_DType_LGBM(test)

# Model and fitting
LGBM = lgb.LGBMRegressor()
LGBM.fit(x_train, y_train)

# Predict test-data-set
pred_test_LGBM = LGBM.predict(test)

# remove negative values
for i in range(len(pred_test_LGBM)):
    if pred_test_LGBM[i] < 0.0:
        print(i)
        pred_test_LGBM[i] = 0.0



In [ ]:
#write the predicition to file
writeResultToFile(stores_test, pred_test_LGBM, "LGBM_02_86_precentile_storeType_lat_long_chain_mall_plaace_hier_id")

# Verify format of submission file
submissionVery = pd.read_csv('submissionFiles/LGBM_02_86_precentile_storeType_lat_long_chain_mall_plaace_hier_id.csv')
submissionVery.info()

## _____ Random Forest Regressor _____

Load, preprocess and convert data to correct format

In [ ]:
# Load training and test data
stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')

# Preprocess/Clean data
remove_columns(stores_train, ['store_id','year','store_name','sales_channel_name','address','chain_name','mall_name'])
remove_columns(stores_test, ['store_id','year','store_name','sales_channel_name','address','chain_name','mall_name'])
#remove_retailers_with_0_revenue(stores_train)
quantile_storeType_vs_revenue(stores_train,0.10, 0.80)

# Divide data into x and y train
x_train = stores_train.drop('revenue', axis=1)
y_train = stores_train['revenue']
x_test = stores_test.copy()

# Convert from object type to numerical
#train set
cat_columns = x_train.select_dtypes(['object']).columns
x_train[cat_columns] = x_train[cat_columns].apply(lambda x: pd.factorize(x)[0])
#test set
cat_columns = x_test.select_dtypes(['object']).columns
x_test[cat_columns] = x_test[cat_columns].apply(lambda x: pd.factorize(x)[0])



In [ ]:
stores_train.head()

Train model

In [ ]:
# Model
RFR = RandomForestRegressor(n_estimators=100)

# Fitting
RFR.fit(x_train, y_train)


Test RFR model

In [ ]:
# predicting the training data set as a pin pointer
pred_train_RFR = RFR.predict(x_train)
print(rmsle(y_train, pred_train_RFR))
print(RFR.score(x_train, y_train))

Predict test and submit

In [ ]:
pred_test_RFR = RFR.predict(x_test)

In [ ]:
# Write to file
writeResultToFile(stores_test, pred_test_RFR, "RFR_10_80_percentile")

# Verify format of submission file
submissionVery = pd.read_csv('submissionFiles/RFR_10_80_percentile.csv')
submissionVery.info()

## Emil modeller

### model 1


In [ ]:
# pythons stuff emil

# 3. Feature Engineering

### some feature moding

In [ ]:
#modding data....

# 4. Model Interpretations

### Lime

In [ ]:
#lime stuff in python

### Feature importance

In [ ]:
#feature importance

### PDP

In [ ]:
#PDP

# 5. Final improved models/predictions

### model 1

In [ ]:
#final model 1

### model 2

In [ ]:
#final model 2

# Testing


#### RMSLE

In [ ]:
def rmsle(y_true, y_pred):
    return metrics.mean_squared_log_error(y_true, y_pred)**0.5


### Optuna

optuna code is copied from: https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

In [ ]:
stores_train = pd.read_csv('data/stores_train.csv')

# select prefered columns
remove_columns(stores_train, ['store_id','year','store_name','sales_channel_name','grunnkrets_id','address'])

# Divide data into train and test set
#temp_x = stores_train.drop('revenue', axis=1)
#temp_y = stores_train['revenue']

#_, x_test, _, y_true = train_test_split(temp_x, temp_y, test_size=0.20, random_state=1)


# Preprocess/Clean data
quantile_storeType_vs_revenue(stores_train,0.02, 0.86)
#remove_retailers_with_0_revenue(stores_train)

# Divide data into x and y train, and test data for submission
x_train = stores_train.drop('revenue', axis=1)
y_train = stores_train['revenue']


# Convert from object type to numerical
x_train = convert_DType_LGBM(x_train)


In [ ]:
def objective(trial,x_train=x_train,y_train=y_train):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(x_train, y_train)):
        X_train, X_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
        y_train, y_test = y_train[train_idx], y_train[test_idx]

        model = lgb.LGBMRegressor(**param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)]
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)


In [ ]:
study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial)
study.optimize(func, n_trials=20)

In [ ]:
optuna.visualization.plot_param_importances(study)
